In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [2]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [3]:
soup_corona = bs(html_corona, 'html.parser')

In [4]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [5]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewCases"].astype("float")
corona_df["NewDeaths"].replace("\,","",regex = True, inplace = True)
corona_df["NewDeaths"].replace("\+","", regex = True, inplace = True)
corona_df["NewDeaths"] = corona_df["NewDeaths"].astype("float")


#add date column for date data scrped
corona_df["Date"]= dt.datetime.today()

In [6]:
corona_df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Date
0,World,3190584,54076.0,226284.0,8471.0,988067.0,1976233,60419.0,409.00,29.0,0.0,0.0,2020-04-29 15:24:49.986126
1,USA,1049431,13666.0,60640.0,1374.0,144411.0,844380,19110.0,3170.00,183.0,6043315.0,18258.0,2020-04-29 15:24:49.986126
2,Spain,236899,4771.0,24275.0,453.0,132929.0,79695,7764.0,5067.00,519.0,1414477.0,30253.0,2020-04-29 15:24:49.986126
3,Italy,203591,2086.0,27682.0,323.0,71252.0,104657,1795.0,3367.00,458.0,1910761.0,31603.0,2020-04-29 15:24:49.986126
4,France,165911,0.0,24087.0,427.0,48228.0,93596,4387.0,2542.00,369.0,463662.0,7103.0,2020-04-29 15:24:49.986126
...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Caribbean Netherlands,5,0.0,0.0,0.0,0.0,5,0.0,191.00,0.0,110.0,4195.0,2020-04-29 15:24:49.986126
209,Anguilla,3,0.0,0.0,0.0,3.0,0,0.0,200.00,0.0,0.0,0.0,2020-04-29 15:24:49.986126
210,Saint Pierre Miquelon,1,0.0,0.0,0.0,0.0,1,0.0,173.00,0.0,0.0,0.0,2020-04-29 15:24:49.986126
211,Yemen,1,0.0,0.0,0.0,1.0,0,0.0,0.03,0.0,120.0,4.0,2020-04-29 15:24:49.986126


In [7]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)
#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)
df_used["ClosedCases"] = df_used["TotalDeaths"] +  df_used["TotalRecovered"]

df_used["ClosedDeathRate"] = df_used["TotalDeaths"]/df_used["ClosedCases"]
df_used["ClosedDeathRate"].fillna(0,inplace = True)

df_used["CurrentDeathRate"] = df_used["TotalDeaths"]/df_used["TotalCases"]
df_used["CurrentDeathRate"].fillna(0,inplace = True)

total_cases = df_used["TotalCases"].sum()
total_closed_cases = df_used["ClosedCases"].sum()

df_used["Weight"] = [x/total_closed_cases for x in df_used["ClosedCases"]]

# #checking if total weight is 1 
# validate_weight = df_used["Weight"].sum()


df_used["DeathRateXWeight"] = df_used["CurrentDeathRate"] *df_used["Weight"]
df_used["DeathRateXWeight"].sum()
df_used["notes"] = ["Last 8 columns are added by Roaya" for x in df_used["DeathRateXWeight"]  ]
df_used.sort_values(by = "DeathRateXWeight", ascending = False)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Deaths/1M pop,...,Tests/ 1M pop,Date,Longitude,Latitude,ClosedCases,ClosedDeathRate,CurrentDeathRate,Weight,DeathRateXWeight,notes
1,Spain,236899,4771.0,24275.0,453.0,132929.0,79695,7764.0,5067.0,519.0,...,30253.0,2020-04-29 15:24:49.986126,-3.647550,40.244487,157204.0,0.154417,0.102470,0.129845,0.013305,Last 8 columns are added by Roaya
2,Italy,203591,2086.0,27682.0,323.0,71252.0,104657,1795.0,3367.0,458.0,...,31603.0,2020-04-29 15:24:49.986126,12.070013,42.796626,98934.0,0.279803,0.135969,0.081716,0.011111,Last 8 columns are added by Roaya
0,USA,1049431,13666.0,60640.0,1374.0,144411.0,844380,19110.0,3170.0,183.0,...,18258.0,2020-04-29 15:24:49.986126,-97.922211,39.381266,205051.0,0.295731,0.057784,0.169365,0.009787,Last 8 columns are added by Roaya
3,France,165911,0.0,24087.0,427.0,48228.0,93596,4387.0,2542.0,369.0,...,7103.0,2020-04-29 15:24:49.986126,2.618787,47.824905,72315.0,0.333084,0.145180,0.059730,0.008672,Last 8 columns are added by Roaya
8,Iran,93657,1073.0,5957.0,80.0,73791.0,13909,2965.0,1115.0,71.0,...,5398.0,2020-04-29 15:24:49.986126,54.274070,32.575033,79748.0,0.074698,0.063604,0.065869,0.004190,Last 8 columns are added by Roaya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,Vietnam,270,0.0,0.0,0.0,222.0,48,8.0,3.0,0.0,...,2188.0,2020-04-29 15:24:49.986126,106.299147,16.646017,222.0,0.000000,0.000000,0.000183,0.000000,Last 8 columns are added by Roaya
141,Mozambique,76,0.0,0.0,0.0,12.0,64,0.0,2.0,0.0,...,54.0,2020-04-29 15:24:49.986126,35.533675,-17.273816,12.0,0.000000,0.000000,0.000010,0.000000,Last 8 columns are added by Roaya
139,Uganda,79,0.0,0.0,0.0,52.0,27,0.0,2.0,0.0,...,600.0,2020-04-29 15:24:49.986126,32.369080,1.274693,52.0,0.000000,0.000000,0.000043,0.000000,Last 8 columns are added by Roaya
127,Madagascar,128,0.0,0.0,0.0,90.0,38,1.0,5.0,0.0,...,112.0,2020-04-29 15:24:49.986126,46.704737,-19.371896,90.0,0.000000,0.000000,0.000074,0.000000,Last 8 columns are added by Roaya


In [8]:
df_used.to_csv('data.csv')
df_used.to_html('data.html')

In [9]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [10]:
corona_json

'[{"Country":"USA","TotalCases":1049431,"NewCases":13666.0,"TotalDeaths":60640.0,"NewDeaths":1374.0,"TotalRecovered":144411.0,"ActiveCases":844380,"CriticalCases":19110.0,"Tot\\u00a0Cases\\/1M pop":3170.0,"Deaths\\/1M pop":183.0,"TotalTests":6043315.0,"Tests\\/ 1M pop":18258.0,"Date":1588173889986,"Longitude":-97.922211,"Latitude":39.381266,"ClosedCases":205051.0,"ClosedDeathRate":0.2957313059,"CurrentDeathRate":0.0577836942,"Weight":0.1693649568,"DeathRateXWeight":0.0097865329,"notes":"Last 8 columns are added by Roaya"},{"Country":"Spain","TotalCases":236899,"NewCases":4771.0,"TotalDeaths":24275.0,"NewDeaths":453.0,"TotalRecovered":132929.0,"ActiveCases":79695,"CriticalCases":7764.0,"Tot\\u00a0Cases\\/1M pop":5067.0,"Deaths\\/1M pop":519.0,"TotalTests":1414477.0,"Tests\\/ 1M pop":30253.0,"Date":1588173889986,"Longitude":-3.64755047,"Latitude":40.24448698,"ClosedCases":157204.0,"ClosedDeathRate":0.1544171904,"CurrentDeathRate":0.1024698289,"Weight":0.1298450077,"DeathRateXWeight":0.01